In [4]:
%%writefile mapper.py

from __future__ import division
import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        article_id = int(article_id)
    except ValueError as e:
        continue
        
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    words = [x.lower() for x in words if x.lower() not in stop_words]
    words_count = Counter(words)
    N = len(words)
    
    for word, count in words_count.items():
        tf = count / N
        print "%s\t%d\t%f" % (word, article_id, tf)

Overwriting mapper.py


In [5]:
%%writefile reducer.py

from __future__ import division
import sys
from math import log

cur_word = None

for line in sys.stdin:
    
    try:
        word, article_id, tf = line.strip().split('\t')
        article_id, tf = int(article_id), float(tf)
    except ValueError as e:
        continue
        
    if cur_word != word:
        if cur_word:
            idf = 1 / log(1 + Dt)
            
            for a_id, a_tf in article_tf:
                tfidf = a_tf * idf
                print "%s\t%d\t%f" % (cur_word, a_id, tfidf)
        
        Dt = 0
        article_tf = []
        cur_word = word
    
    Dt += 1
    article_tf.append((article_id, tf))
    
if cur_word:
    idf = 1 / log(1 + Dt)
    
    for a_id, a_tf in article_tf:
        tfidf = a_tf * idf
        print "%s\t%d\t%f" % (cur_word, a_id, tfidf)

Overwriting reducer.py


In [6]:
%%bash

OUT_DIR="TFIDF_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming TF-IDF" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python2 mapper.py" \
    -reducer "python2 reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* | grep -P 'labor\t12\t' | cut -f3

Couldn't find program: 'bash'
